# Recessions

Note: Recessions cannot really be determined algorithmically. Many factors need to be considered. Nonetheless ...

## Set-up

In [1]:
import pandas as pd
import readabs as ra
from IPython.display import display
from pandas import DataFrame
from mgplot import clear_chart_dir, finalise_plot, set_chart_dir, line_plot

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# save charts in this notebook
CHART_DIR = "./CHARTS/Recessions/"
set_chart_dir(CHART_DIR)
clear_chart_dir()

## Get the main data items

In [3]:
# constants
CAT_MDB, CAT_GDP = "1364.0.15.003", "5206.0"


def get_data() -> tuple[dict[str, pd.Series], pd.DataFrame]:
    """Get a dictionary of data items from the ABS."""

    aggregates = "5206001_Key_Aggregates"
    wanted = {
        # "Series ID": ["Category ID", "single-excel-only table name", "Short Series Title"],
        # These are the series we want to extract from the ABS Modeller's database
        # All are Seasonally adjusted. GDP is also in Chain volume measures.
        "A2454517C": [CAT_MDB, "", "Labour force"],
        "A2454521V": [CAT_MDB, "", "Unemployed"],
        "A2454489F": [
            CAT_MDB,
            "",
            "GDP (SA/CVM/MDB)",
        ],  # Chain volume measures, seasonally adjusted
        # we use the original series from the National Accounts to derive the population
        # all the way back to 1959. The GDP per capita SA/CVM stats only go back to 1973.
        "A2302460K": [CAT_GDP, aggregates, "GDP per capita (O/CVM/KA)"],
        "A2302459A": [CAT_GDP, aggregates, "GDP (O/CVM/KA)"],
    }

    data, meta = {}, {}
    for series_id, (category_id, seo, title) in wanted.items():
        d, m = ra.read_abs_series(category_id, series_id, single_excel_only=seo)
        data[title] = d[series_id]
        meta[title] = m.loc[series_id]
    return data, pd.DataFrame(meta).T

### Calculations

In [4]:
def build_dataset() -> tuple[DataFrame, float, dict[str, set[str]]]:
    """Build a dataset from the ABS data."""

    def two_negative_quarters(series):
        """Identify two consecutive negative quarters."""
        return (series < 0) & ((series.shift(-1) < 0) | (series.shift(1) < 0))

    sources: dict[str, set[str]] = {}
    data_dict, meta = get_data()
    print("Data captured from ABS:")
    display(meta)
    data = pd.DataFrame(data_dict)

    data["population"] = data["GDP (O/CVM/KA)"] / data["GDP per capita (O/CVM/KA)"]
    sources["population"] = set([CAT_GDP])

    data["Employed"] = data["Labour force"] - data["Unemployed"]
    sources["Employed"] = set([CAT_MDB])

    data["Employment Growth Q/Q"] = data["Employed"].pct_change(1) * 100
    sources["Employment Growth Q/Q"] = sources["Employed"]

    data["Employment Growth Y/Y"] = data["Employed"].pct_change(4) * 100
    sources["Employment Growth Y/Y"] = sources["Employed"]

    data["Employment Technical Recession"] = two_negative_quarters(
        data["Employment Growth Q/Q"]
    )
    sources["Employment Technical Recession"] = sources["Employed"]

    data["Negative Annual Employment Growth"] = data["Employment Growth Y/Y"] < 0
    sources["Negative Annual Employment Growth"] = sources["Employed"]

    data["Unemployment Rate"] = data["Unemployed"] / data["Labour force"] * 100
    sources["Unemployment Rate"] = set([CAT_MDB])

    data["GDP Growth Q/Q"] = data["GDP (SA/CVM/MDB)"].pct_change(1) * 100
    sources["GDP Growth Q/Q"] = set([CAT_MDB])

    data["GDP Growth Y/Y"] = data["GDP (SA/CVM/MDB)"].pct_change(4) * 100
    sources["GDP Growth Y/Y"] = set([CAT_MDB])

    data["Negative Annual GDP Growth"] = data["GDP Growth Y/Y"] < 0
    sources["Negative Annual GDP Growth"] = set([CAT_MDB])

    data["GDP Technical Recession"] = two_negative_quarters(data["GDP Growth Q/Q"])
    sources["GDP Technical Recession"] = sources["GDP Growth Q/Q"]

    data["GDP Per Capita"] = data["GDP (SA/CVM/MDB)"] / data["population"]
    sources["GDP Per Capita"] = sources["population"] | sources["GDP Growth Q/Q"]

    data["GDP per Capita Growth"] = data["GDP Per Capita"].pct_change(1) * 100
    sources["GDP per Capita Growth"] = sources["GDP Per Capita"]

    data["GDP per Capita Technical Recession"] = two_negative_quarters(
        data["GDP per Capita Growth"]
    )
    sources["GDP per Capita Recession"] = sources["GDP per Capita Growth"]

    # unemployment growth exceeds a threshold
    threshold = 0.75  # percentage points - akin to the Sahm Rule
    data["Rapid Unemployment Growth"] = (
        data["Unemployment Rate"].rolling(4).min().shift(1)
        < data["Unemployment Rate"] - threshold
    )
    return data, threshold, sources


DATA, THRESHOLD, SOURCES = build_dataset()

Data captured from ABS:


,Data Item Description,Series Type,Series ID,Series Start,Series End,No. Obs.,Unit,Data Type,Freq.,Collection Month,Table,Table Description,Catalogue number
Labour force,Total labour force ;,Seasonally Adjusted,A2454517C,1959-09-01 00:00:00,2025-03-01 00:00:00,263,000,DERIVED,Quarter,3,1364015003,Tables 01 to 17,1364.0.15.003
Unemployed,Total unemployed ;,Seasonally Adjusted,A2454521V,1959-09-01 00:00:00,2025-03-01 00:00:00,263,000,DERIVED,Quarter,3,1364015003,Tables 01 to 17,1364.0.15.003
GDP (SA/CVM/MDB),Gross domestic product (Chain volume measures) ;,Seasonally Adjusted,A2454489F,1959-09-01 00:00:00,2025-03-01 00:00:00,263,$ Millions,DERIVED,Quarter,3,1364015003,Tables 01 to 17,1364.0.15.003
GDP per capita (O/CVM/KA),GDP per capita: Chain volume measures ;,Original,A2302460K,1959-09-01 00:00:00,2025-06-01 00:00:00,264,$,DERIVED,Quarter,3,5206001_Key_Aggregates,Key National Accounts Aggregates,5206.0
GDP (O/CVM/KA),Gross domestic product: Chain volume measures ;,Original,A2302459A,1959-09-01 00:00:00,2025-06-01 00:00:00,264,$ Millions,DERIVED,Quarter,3,5206001_Key_Aggregates,Key National Accounts Aggregates,5206.0


/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_78117/1189032085.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data["Employment Growth Q/Q"] = data["Employed"].pct_change(1) * 100
/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_78117/1189032085.py:23: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data["Employment Growth Y/Y"] = data["Employed"].pct_change(4) * 100
/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_78117/1189032085.py:37: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Ei

## Plot

### Utilities for plotting

In [5]:
def highlight(ax, series, color, alpha=0.5, label=None) -> None:
    """Add highlights to a chart based on a Boolean series."""

    o_series = series.copy()
    o_series.index = [p.ordinal for p in o_series.index]

    shading, start, previous = False, None, None
    for index, item in o_series.items():
        if item and not shading:
            shading, start = True, index
        if shading and not item:
            ax.axvspan(start, previous, color=color, alpha=alpha, label=label)
            shading = False
            label = None
        previous = index
    if shading:
        ax.axvspan(start, previous, color=color, alpha=alpha, label=label)

In [6]:
def rfooter(label: str) -> str:
    """Generate text for the right footer."""
    return f'ABS {", ".join(SOURCES[label])}'

In [7]:
# constants
R_COLOUR, ALPHA = "darkorange", 0.5
LFOOTER = "Australia. "
SHOW = False
COMMON = {
    "y0": True,
    "show": SHOW,
    "legend": True,
}

### Indicators of a (potential) recession

In [8]:
def chart_exents(
    data: DataFrame,
    series: str,  # column name in data, column for whom a line will be plotted
    event: str,  # column name in data, column to highlight
) -> None:
    """Plot data comprising a line, and shaded zones for important events."""

    ax = line_plot(data[series], color="darkblue", label_series=True,)
    highlight(
        ax,
        data[event],
        color=R_COLOUR,
        alpha=ALPHA,
        label=event,
    )

    tr = "Technical recession is 2+ quarters of negative growth. "
    gdp = "GDP growth is seasonally adjusted, chain volume measures. "
    finalise_plot(
        ax,
        title=f"{series} - {event}",
        ylabel="Per cent",
        xlabel=None,
        rfooter=rfooter(series),
        lfooter=f"{LFOOTER}"
        + (tr if "Recession" in event else "")
        + (gdp if "GDP" in event else ""),
        **COMMON,
    )

In [9]:
def plot_events(data: DataFrame = DATA) -> None:
    """Plot GDP growth and technical recessions."""

    items = [
        # data, recession
        ("GDP Growth Q/Q", "GDP Technical Recession"),
        ("GDP Growth Y/Y", "Negative Annual GDP Growth"),
        ("Employment Growth Q/Q", "Employment Technical Recession"),
        ("Employment Growth Y/Y", "Negative Annual Employment Growth"),
        ("GDP per Capita Growth", "GDP per Capita Technical Recession"),
        ("Unemployment Rate", "Rapid Unemployment Growth"),
    ]

    for series, important in items:
        chart_exents(data, series, important)


plot_events()

### Recession = Any three of the above

In [10]:
def plot_recessions(data: DataFrame = DATA) -> None:
    """Plot Australian recessions based on 3 or more of the
    above indicators, where a recession indicates an extended
    period of economic contraction and an associated decrease
    in the number employed and an increase in the unemployment
    rate."""

    indicator_set = [
        "GDP Technical Recession",
        "Negative Annual GDP Growth",
        "GDP per Capita Technical Recession",
        "Rapid Unemployment Growth",
        "Employment Technical Recession",
        "Negative Annual Employment Growth",
    ]

    recession_threshold = 3
    shoulder_threshold = 1
    recession_points = data[indicator_set].sum(axis=1)
    recession = (
        # three or more indicators
        (recession_points >= recession_threshold)
        # plus one quarter with one or more indicators (shoulder)
        | (
            (recession_points >= shoulder_threshold)
            & (recession_points.shift(1) >= recession_threshold)
        )
        | (
            (recession_points >= shoulder_threshold)
            & (recession_points.shift(-1) >= recession_threshold)
        )
    )
    print(
        f"Naive recession probability: {recession.sum() / len(recession) * 100:0.0f}%"
    )

    # chart recession indicator intensity
    recession_points.name = "Indicator intensity"
    ax = line_plot(recession_points, label_series=True, color=["darkblue", "darkred"])
    highlight(ax, recession, color=R_COLOUR, alpha=ALPHA, label="Recessions")
    finalise_plot(
        ax,
        title="Australian Recessions (by indicator intensity)",
        ylabel="Indicator count",
        rfooter=rfooter("GDP per Capita Growth"),
        lfooter=f"{LFOOTER} "
        + f"Indicator threshold is {recession_threshold} of {len(indicator_set)}. ",
         **COMMON,
    )

    # --- plot recessions agains GDP and unemployment data
    ax = line_plot(
        data[["Employment Growth Y/Y", "GDP Growth Y/Y"]], 
        color=['red', 'darkblue'],
        width=[1.5, 1.5],
        style=['-', '--'],
        label_series=True,
    )
    highlight(ax, recession, color=R_COLOUR, alpha=ALPHA, label="Recessions")
    finalise_plot(
        ax,
        title="Australian Recessions",
        ylabel="Per cent",
        rfooter=rfooter("GDP per Capita Growth"),
        lfooter=f"{LFOOTER} "
        "GDP is seasonally adjusted, chain volume measures. "
        + f"Indicator threshold is {recession_threshold} of {len(indicator_set)}. ",
        **COMMON,
    )


plot_recessions()

Naive recession probability: 13%


## Watermark

In [11]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-09-04 08:24:21

Python implementation: CPython
Python version       : 3.13.6
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

readabs: 0.1.4
IPython: 9.4.0
pandas : 2.3.1
mgplot : 0.2.12

Watermark: 2.5.0

